# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os.path as op
import pickle

# Data Fetching

In [2]:
A1=np.empty((0,5),dtype='float32')
U1=np.empty((0,7),dtype='float32')
node=['150','149','147','144','142','140','136','61']
mon=['Apr','Mar','Aug','Jun','Jul','Sep','May','Oct']
for j in node:
  for i in mon:
    inp= pd.read_csv('../../../data_gkv/AT510_Node_'+str(j)+'_'+str(i)+'19_OutputFile.csv',usecols=[1,2,3,15,16],low_memory=False)
    out= pd.read_csv('../../../data_gkv/AT510_Node_'+str(j)+'_'+str(i)+'19_OutputFile.csv',usecols=[5,6,7,8,17,18,19],low_memory=False)
    
    inp=np.array(inp,dtype='float32')
    out=np.array(out,dtype='float32')
    
    A1=np.append(A1, inp, axis=0)
    U1=np.append(U1, out, axis=0)

print(A1)
print(U1)


[[1.50000e+02 1.90401e+05 7.25000e+02 2.75500e+01 8.03900e+01]
 [1.50000e+02 1.90401e+05 8.25000e+02 2.75600e+01 8.03300e+01]
 [1.50000e+02 1.90401e+05 9.25000e+02 2.75800e+01 8.02400e+01]
 ...
 [6.10000e+01 1.91020e+05 1.94532e+05 2.93700e+01 7.52100e+01]
 [6.10000e+01 1.91020e+05 1.94632e+05 2.93500e+01 7.52700e+01]
 [6.10000e+01 1.91020e+05 1.94732e+05 2.93400e+01 7.53000e+01]]
[[ 28.     3.   -52.   ...  16.97  19.63  20.06]
 [ 28.    15.   -53.   ...  16.63  19.57  23.06]
 [ 31.    16.   -55.   ...  17.24  19.98  20.24]
 ...
 [ 76.    12.   -76.   ...   3.47   3.95   4.35]
 [ 75.    13.   -76.   ...   3.88   4.33   4.42]
 [ 76.    12.   -75.   ...   3.46   4.07   4.28]]


# A fast algorithm for Independent Component Analysis Decomposition

In [3]:
from sklearn.decomposition import PCA
import warnings
scaler_obj1=PCA()
scaler_obj2=PCA()
X1=scaler_obj1.fit_transform(A1)
Y1=scaler_obj2.fit_transform(U1)

warnings.filterwarnings(action='ignore', category=UserWarning)

# Parameter Tuning

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV

def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [7,8,10,],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'n_estimators' : [80,90,100,200,400,1000],
        'objective': ['reg:squarederror','count:poisson']
    }

    xgb_model = XGBRegressor(tree_method='gpu_hist', gpu_id=0)

    gsearch = RandomizedSearchCV(estimator = xgb_model, param_distributions = param_tuning, n_iter = 5, cv = 2, verbose=2, random_state=0, n_jobs = -1)

    
    grid_result = MultiOutputRegressor(gsearch).fit(x_train, y_train)

    return grid_result.estimators_[0].best_params_

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=0)

params=hyperParameterTuning(x_train,y_train)
params

# Model

In [4]:
# Splitting Data into training and testing dataset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=42)

from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

#creating object of sgboostregressor
model1=MultiOutputRegressor(XGBRegressor(tree_method='gpu_hist', gpu_id=0,max_depth=8,cosample_bytree=.5,learning_rate=.1,min_child_weight=3,
                   n_estimators=200,subsample=.7))

#training the model
model_fit1=model1.fit(x_train, y_train)
print("Model training is Done!!")

#saving as file
filename1 = 'xgboost.sav'
pickle.dump(model_fit1, open(filename1, 'wb'))

[09:47:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "cosample_bytree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:48:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "cosample_bytree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:48:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "cosample_bytree" } might not be use

# Error Analysis

In [5]:
from sklearn.metrics import r2_score
from sklearn import metrics
train_sizes=['NO2','O3','NO','CO','PM1','PM2.5','PM10']

#finding out the r2 score
y_train_pred1=model1.predict(x_train)
r2_train1=r2_score(y_train,y_train_pred1,multioutput='variance_weighted')

y_test_pred1=model1.predict(x_test)
r2_test1=r2_score(y_test,y_test_pred1,multioutput='variance_weighted')

print('r2 score on train data '+str(r2_train1))
print('r2 score on test data '+ str(r2_test1))

xgboost_mae=metrics.mean_absolute_error(y_test, y_test_pred1)
xgboost_mse=metrics.mean_squared_error(y_test, y_test_pred1)
xgboost_rmse=np.sqrt(xgboost_mse)
print('Mean Absolute Error:',xgboost_mae)
print('Mean Squared Error:',xgboost_mse )
print('Root Mean Squared Error:',xgboost_rmse)
print(' \n')

r2 score on train data 0.9391336619957393
r2 score on test data -43.845155060127134
Mean Absolute Error: 71.18526
Mean Squared Error: 64184236.0
Root Mean Squared Error: 8011.5063
 



# Prediction of particles

In [6]:
import pickle
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=42)
loaded_model_fit7 = pickle.load(open("xgboost.sav", 'rb'))
y_test_pred=loaded_model_fit7.predict(x_test)
print("Predicted :\n",y_test_pred)
print("\n")
r2_test=r2_score(y_test,y_test_pred,multioutput='variance_weighted')
print("R2 Score : ",r2_test)

Predicted :
 [[ 4.5720367e+00 -6.8629669e+01  1.2030648e+02 ... -3.1364040e+01
  -6.4518857e-01  2.3640305e-01]
 [-1.0241197e+02 -5.2321771e+02 -3.5265011e+01 ... -2.4913519e+01
   4.5548695e-01  4.8830602e-03]
 [-4.4976200e+01 -4.5303303e+01 -3.7054486e+02 ... -3.0610549e+00
  -3.3596101e-01  4.4598780e-03]
 ...
 [-8.2302925e+01 -2.2591805e+01 -5.0545932e+02 ...  5.4868584e+01
  -1.1703753e+00 -4.9131614e-01]
 [-4.4086311e+01  3.1535965e+01  4.0011691e+02 ... -2.2299282e+01
  -1.3108271e+00 -2.1083799e-01]
 [-6.1457905e+01 -5.5224068e+01  4.1717386e+02 ... -2.2293032e+01
   3.3821380e+00  2.8112048e-01]]


R2 Score :  -43.845155060127134


In [7]:
x_test=scaler_obj1.inverse_transform(x_test)
y_test_pred=scaler_obj2.inverse_transform(y_test_pred)
y_test=scaler_obj2.inverse_transform(y_test)
pd.DataFrame(y_test_pred)

,0,1,2,3,4,5,6
0,-17.659168,95.158188,-82.162498,558.172546,2.484315,2.806837,3.270098
1,-471.405243,-15.111542,-4.500454,402.444092,5.331635,6.042616,8.191634
2,6.200203,45.778851,-25.690651,67.331726,16.592995,19.408037,21.306499
3,64.816086,10.991188,-43.882835,842.880493,24.371634,27.862608,32.280704
4,20.327467,31.058437,4.190800,-162.304169,4.394006,5.297158,6.422619
...,...,...,...,...,...,...,...
432566,23.807953,35.333183,63.572655,-10.052124,6.572509,7.610014,9.472013
432567,85.420128,44.052616,-84.388199,416.274902,2.316730,2.776682,3.477535
432568,29.228991,8.617058,-48.759842,-67.578400,46.212147,54.000237,57.113670
432569,82.767799,47.225719,1093.367432,838.086609,7.651624,8.195186,8.500440


In [8]:
import datetime
from datetime import datetime
temp_arr=x_test[0:,1]
dates=list()
for i in range(0,len(temp_arr)):
    datetime_str=str(int(temp_arr[i]))
    datetime_str=datetime_str[0:6]
    if(datetime_str[4:6]=="00"):
        datetime_str=datetime_str[0:4]+'1'
    datetime_obj = datetime.strptime(datetime_str,"%y%m%d")
    dates.append(str(datetime_obj.date()))


In [9]:
import pandas as pd
df1=pd.DataFrame(y_test,index=dates,columns=['NO2','O3','NO','CO','PM1','PM2.5','PM10'])
df1.index.name="DATE"
print("Actual Values:")
df1

Actual Values:


,NO2,O3,NO,CO,PM1,PM2.5,PM10
DATE,,,,,,,
2019-06-04,81.999992,35.999981,-79.000076,706.000061,1.999994,2.420012,2.819992
2019-09-26,19.000008,50.000031,-7.999964,478.000000,10.170000,10.950006,12.669997
2019-05-14,58.000000,38.000008,-33.999943,-107.000092,27.400002,31.749990,41.180004
2019-04-18,82.999985,4.999985,-27.000103,1580.000244,39.410004,45.179993,50.160015
2019-08-31,5.000015,54.000038,-3.999901,-170.000153,6.900000,8.250006,9.109994
...,...,...,...,...,...,...,...
2019-03-29,-2.999992,104.000038,64.000160,28.999969,7.130006,7.980010,8.190001
2019-06-30,76.999992,39.999985,-81.000046,413.000000,1.939995,2.610010,4.119987
2019-10-02,21.000006,3.000038,-47.999969,-3.000000,52.910011,61.519974,67.230019


In [10]:
arr=["Y_Actual"]*25
temp_df1=df1.head(25)
temp_df1['Data']=arr
temp_df1.to_excel("xgboost_y_test.xlsx")
temp_df1

C:\Users\utkar\AppData\Local\Temp/ipykernel_14236/2545994415.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df1['Data']=arr


,NO2,O3,NO,CO,PM1,PM2.5,PM10,Data
DATE,,,,,,,,
2019-06-04,81.999992,35.999981,-79.000076,706.000061,1.999994,2.420012,2.819992,Y_Actual
2019-09-26,19.000008,50.000031,-7.999964,478.000000,10.170000,10.950006,12.669997,Y_Actual
2019-05-14,58.000000,38.000008,-33.999943,-107.000092,27.400002,31.749990,41.180004,Y_Actual
2019-04-18,82.999985,4.999985,-27.000103,1580.000244,39.410004,45.179993,50.160015,Y_Actual
2019-08-31,5.000015,54.000038,-3.999901,-170.000153,6.900000,8.250006,9.109994,Y_Actual
2019-06-30,10.000008,34.000034,35.000114,51.999847,6.890001,8.320008,10.110000,Y_Actual
2019-07-27,24.000006,6.000038,-37.999950,-69.000092,18.180000,22.039999,23.079998,Y_Actual
2019-07-08,71.999992,11.999985,-93.000092,649.000000,3.119994,4.330011,6.959993,Y_Actual
2019-09-07,52.000000,66.000000,-73.000031,243.999954,20.639997,23.179996,23.709995,Y_Actual


In [11]:
import pandas as pd
df2=pd.DataFrame(y_test_pred,index=dates,columns=['NO2','O3','NO','CO','PM1','PM2.5','PM10'])
df2.index.name="DATE"
print("Predicted Values:")
df2

Predicted Values:


,NO2,O3,NO,CO,PM1,PM2.5,PM10
DATE,,,,,,,
2019-06-04,-17.659168,95.158188,-82.162498,558.172546,2.484315,2.806837,3.270098
2019-09-26,-471.405243,-15.111542,-4.500454,402.444092,5.331635,6.042616,8.191634
2019-05-14,6.200203,45.778851,-25.690651,67.331726,16.592995,19.408037,21.306499
2019-04-18,64.816086,10.991188,-43.882835,842.880493,24.371634,27.862608,32.280704
2019-08-31,20.327467,31.058437,4.190800,-162.304169,4.394006,5.297158,6.422619
...,...,...,...,...,...,...,...
2019-03-29,23.807953,35.333183,63.572655,-10.052124,6.572509,7.610014,9.472013
2019-06-30,85.420128,44.052616,-84.388199,416.274902,2.316730,2.776682,3.477535
2019-10-02,29.228991,8.617058,-48.759842,-67.578400,46.212147,54.000237,57.113670


In [12]:
arr=["Y_Predicted"]*25
temp_df2=df2.head(25)
temp_df2['Data']=arr
temp_df2.to_excel("xgboost_y_test_pred.xlsx")
temp_df2

C:\Users\utkar\AppData\Local\Temp/ipykernel_14236/3694748081.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df2['Data']=arr


,NO2,O3,NO,CO,PM1,PM2.5,PM10,Data
DATE,,,,,,,,
2019-06-04,-17.659168,95.158188,-82.162498,558.172546,2.484315,2.806837,3.270098,Y_Predicted
2019-09-26,-471.405243,-15.111542,-4.500454,402.444092,5.331635,6.042616,8.191634,Y_Predicted
2019-05-14,6.200203,45.778851,-25.690651,67.331726,16.592995,19.408037,21.306499,Y_Predicted
2019-04-18,64.816086,10.991188,-43.882835,842.880493,24.371634,27.862608,32.280704,Y_Predicted
2019-08-31,20.327467,31.058437,4.190800,-162.304169,4.394006,5.297158,6.422619,Y_Predicted
2019-06-30,34.060417,39.867191,28.816994,37.977325,6.136455,7.373557,9.702446,Y_Predicted
2019-07-27,34.447765,14.382759,-42.677555,-61.293427,20.143837,23.654119,25.470499,Y_Predicted
2019-07-08,78.346512,16.658356,-97.106079,610.938110,3.453886,4.128635,4.684057,Y_Predicted
2019-09-07,57.562691,34.525951,-69.379791,528.187073,44.398094,50.263199,55.256477,Y_Predicted


In [13]:
#Completed